In [1]:
!pip install scikit-learn==1.3.2 biopython gensim umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [1]:
from Bio import SeqIO
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

kmer_size = 4
vector_size = 256

def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

In [2]:
def circrna_to_vec(circrna_sequence, k=kmer_size):
  vec = np.zeros(vector_size)
  kmers = circrna_to_kmers(circrna_sequence, k=k)
  for kmer in kmers:
    if kmer in w2v_model.wv:
      vec = vec + w2v_model.wv[kmer]
  return vec


In [3]:
w2v_model = Word2Vec.load("word2vec_model_maize_4mer.bin")

In [4]:
circ_df = pd.read_excel('maize_db.xlsx')

In [5]:
list = circ_df['seq']

columns = [f'wc_4mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-5-1cc24255fe7b>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_4mer_1,wc_4mer_2,wc_4mer_3,wc_4mer_4,wc_4mer_5,wc_4mer_6,wc_4mer_7,wc_4mer_8,wc_4mer_9,wc_4mer_10,...,wc_4mer_247,wc_4mer_248,wc_4mer_249,wc_4mer_250,wc_4mer_251,wc_4mer_252,wc_4mer_253,wc_4mer_254,wc_4mer_255,wc_4mer_256
0,-4.842847,7.479475,12.613859,-9.664886,-0.810559,29.980982,-41.601975,18.577792,-9.856197,2.732827,...,17.032648,40.126735,-14.553852,-4.243147,-10.870053,-11.795699,-10.544674,47.691434,-11.942805,-16.797678
1,73.295406,-62.290622,41.835723,175.268988,-79.680520,-126.594082,306.633031,-267.593629,-29.785972,185.518146,...,-46.820271,-79.350465,-9.566307,25.296402,-78.377040,-23.443641,-79.692286,-56.034896,58.819192,-52.494107
2,-32.675780,32.153700,-6.600949,48.231775,35.962642,17.149901,-37.772983,36.696900,31.526608,2.116354,...,33.311082,-13.926878,7.712176,-7.269773,19.971366,-29.585031,1.523294,14.665458,21.508492,4.491334
3,-1.368031,22.936400,-3.136681,-5.970814,-12.719445,30.691018,-31.731565,-5.908737,-19.326244,-7.547226,...,-14.562025,11.809896,-4.364715,14.992254,27.773269,-26.672485,30.670639,17.201796,5.665467,7.967891
4,22.632758,48.035762,48.833816,12.694329,44.949726,14.833411,18.743641,-78.477201,40.940137,13.717707,...,-50.086140,7.042557,71.323520,-26.477156,-75.035556,-46.504361,-42.700965,-22.942764,82.966075,-50.255035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38780,-34.817851,-1.603454,-9.831531,21.078451,17.190499,26.759408,-44.375381,81.546683,50.319549,-52.069265,...,14.084051,4.123945,28.432224,-0.911503,14.395344,-33.521682,22.703023,40.298385,18.434365,5.945080
38781,-15.292624,-0.882159,44.047676,37.821824,-6.295991,7.444920,-33.084452,10.265649,43.696666,-16.597471,...,-11.288301,-32.893305,36.478031,-14.282901,-8.705481,3.454306,12.907764,-27.757609,0.788657,46.592197
38782,-5.566825,-14.279546,-27.777740,-56.450042,87.303854,62.966050,-61.990254,110.780326,108.447138,-90.256145,...,34.133197,-24.298941,44.812781,-10.680627,19.012171,-57.523479,66.192625,29.087843,25.607870,9.236250
38783,-100.136891,-173.321663,16.093652,266.837541,34.683158,-38.659601,18.718207,155.463656,194.877566,-46.719549,...,130.205332,-118.004013,114.657027,-29.129630,53.280105,-74.418111,6.683515,-20.766160,-44.845353,112.205237


In [6]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_4mer_247,wc_4mer_248,wc_4mer_249,wc_4mer_250,wc_4mer_251,wc_4mer_252,wc_4mer_253,wc_4mer_254,wc_4mer_255,wc_4mer_256
0,zma-circ1-Zm00001d002325,2:10317309-10317467_-,Zm00001d002325,Zm00001d002325_T001,-,multipleTissue,2,10317309,10317467,-,...,17.032648,40.126735,-14.553852,-4.243147,-10.870053,-11.795699,-10.544674,47.691434,-11.942805,-16.797678
1,zma-circ2-Zm00001d038675,6:162376852-162378246_+,Zm00001d038675,Zm00001d038675_T004,-,multipleTissue,6,162376852,162378246,+,...,-46.820271,-79.350465,-9.566307,25.296402,-78.377040,-23.443641,-79.692286,-56.034896,58.819192,-52.494107
2,zma-circ3-Zm00001d038163,6:150032431-150032595_+,Zm00001d038163,Zm00001d038163_T001,-,multipleTissue,6,150032431,150032595,+,...,33.311082,-13.926878,7.712176,-7.269773,19.971366,-29.585031,1.523294,14.665458,21.508492,4.491334
3,zma-circ4-Zm00001d049552,4:34381638-34381747_-,Zm00001d049552,Zm00001d049552_T004,-,multipleTissue,4,34381638,34381747,-,...,-14.562025,11.809896,-4.364715,14.992254,27.773269,-26.672485,30.670639,17.201796,5.665467,7.967891
4,zma-circ5-Zm00001d032567,1:230724608-230725226_-,Zm00001d032567,Zm00001d032567_T001,-,multipleTissue,1,230724608,230725226,-,...,-50.086140,7.042557,71.323520,-26.477156,-75.035556,-46.504361,-42.700965,-22.942764,82.966075,-50.255035


In [7]:
new_df.to_excel('maize_w2vec_4mer_dataset.xlsx', index=False)